# HW

Ben Burt

Ben Christensen

Jane Cox

In [8]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import pandas as pd
from scipy import linalg as la
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from scipy.sparse import linalg as spla

# Our Data Set

In [10]:
r = pd.read_csv("/Users/Armen/Desktop/SpringDataProject/ratings.csv",nrows=100000)

In [11]:
def set_up_test(user_id):
    #inidiscriminately chopping up the dataset to get something small
    df = r[r.userId < 200]
    df = df[df.movieId < 2000]
    user_c = CategoricalDtype(sorted(df.userId.unique()), ordered=True)
    movie_c = CategoricalDtype(sorted(df.movieId.unique()), ordered=True)
    row = df.userId.astype(user_c).cat.codes
    col = df.movieId.astype(movie_c).cat.codes
    #this matrix is movies x users
    matrix = csr_matrix((df['rating'], (row, col)) \
                        , shape=(user_c.categories.size, movie_c.categories.size)).todense().T
    
    #take out our lucky user
    user1 = np.ravel(matrix[:,user_id])
    matrix = np.delete(matrix, user_id, axis = 1)

    #we can only train on the movies that the user has seen
    movies_not_seen = np.where(user1 == 0)[0]
    movies_seen = np.where(user1 != 0)[0]
    
    #turn user's rating into binary 'liked' (1) or 'disliked' (-1)
    known_ratings = user1[movies_seen]
    known_ratings[known_ratings <= 3] = -1 #right now the cutoff is 3. could also try 2.5?
    known_ratings[known_ratings > 3] = 1
    
    #replace all the Nan's with 3's.
    matrix[matrix == 0] = 3
    
    return pd.DataFrame(matrix), movies_not_seen, movies_seen, known_ratings

def for_real(user_id):
    #get only those movies with more than 17 ratings (cutting out ~50% of movies)
    merged = r.merge(r.groupby("movieId").size().reset_index(name='count'),
                     how='right', on='movieId')
    df = merged[merged["count"]>17].sort_values(['userId', 'movieId'])
    
    #get our lucky user
    user1 = df[df.userId == user_id]
    print(user1.shape)
    df.drop(labels=df[df.userId == user_id].index, axis=0, inplace=True)
    
    #create a matrix from the ratings list
    user_c = CategoricalDtype(sorted(df.userId.unique()), ordered=True)
    movie_c = CategoricalDtype(sorted(df.movieId.unique()), ordered=True)
    row = df.userId.astype(user_c).cat.codes
    col = df.movieId.astype(movie_c).cat.codes
    
    #this matrix is movies x users
    sparse_matrix = csr_matrix((df['rating'], (row, col)), \
                               shape=(user_c.categories.size, movie_c.categories.size)).T
    
    #we can't deal with these NaN's, so we're lazy and just replace them all with 3's
    dfs = pd.SparseDataFrame(sparse_matrix, \
                         index=movie_c.categories, \
                         columns=user_c.categories, \
                         default_fill_value=3)
    
    #we can only train on the movies that the user has seen
    movies_seen = dfs.loc[user1.movieId.values].index.values
    movies_not_seen = list(set(dfs.index.values) - set(movies_seen))
    
    #turn user's rating into binary 'liked' (1) or 'disliked' (-1)
    known_ratings = user1.rating.values
    known_ratings[known_ratings <= 3] = -1 #right now the cutoff is 3. could also try 2.5?
    known_ratings[known_ratings > 3] = 1
    
    return dfs, movies_not_seen, movies_seen, known_ratings

In [12]:
user_id = 50
#dfs, movies_not_seen, movies_seen, known_ratings = set_up_test(user_id)
dfs, movies_not_seen, movies_seen, known_ratings = for_real(user_id)

       userId  movieId  rating   timestamp  count
170        50       32     4.0  1182349090    220
37346      50       39     3.5  1182349203    146
391        50       47     4.0  1182348347    214
26575      50      150     3.5  1182349064    235
74290      50      215     5.0  1182347856     25
1607       50      296     5.0  1182348326    350
74315      50      308     4.0  1182347603     29
1952       50      318     4.0  1182349053    305
57215      50      319     3.0  1181646332     35
38828      50      344     2.0  1182677668    193
24327      50      356     4.5  1182349043    340
30495      50      357     4.0  1182349181    136
39235      50      372     5.0  1181644438     39
50540      50      497     4.0  1182678132     78
30757      50      539     4.0  1182349170    136
2508       50      541     4.0  1182677784    156
39686      50      551     4.0  1182677995    105
2677       50      589     4.5  1182349075    256
28018      50      590     3.5  1182349067    216


In [13]:
#for the test train split, we only want the movies for which we have labels
x_train, x_test, y_train, y_test = train_test_split(dfs.iloc[movies_seen].values,known_ratings)

#This is what we'll want to predict
x_to_predict = dfs.iloc[movies_not_seen]

IndexError: positional indexers are out-of-bounds

In [ ]:
parameters = {'kernel' : ('rbf', 'poly')
              ,'C' : [1,10], 'degree' : [1,2,3,4], 'gamma' : [1,2,3,'auto']}
model = SVC()
#clf = GridSearchCV(model, parameters)
model.fit(x_train,y_train)
model.score(x_test,y_test)